In [14]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings('ignore')

In [2]:
complete_train_data = pd.read_csv('data/processed/train_proc_1.csv')
complete_test_data = pd.read_csv('data/processed/test_proc_1.csv')

In [9]:
complete_test_data.head()

,air_store_id,visit_date,reserve_visitors,calendar_date,day_of_week,holiday_flg,month,day
0,air_00a91d42b08b08d9,2017-04-23,0.0,2017-04-23,6,0,4,23
1,air_0164b9927d20bcc3,2017-04-23,0.0,2017-04-23,6,0,4,23
2,air_0241aa3964b7f861,2017-04-23,0.0,2017-04-23,6,0,4,23
3,air_0328696196e46f18,2017-04-23,0.0,2017-04-23,6,0,4,23
4,air_034a3d5b40d5b1b1,2017-04-23,0.0,2017-04-23,6,0,4,23


<h2>Feature Engineering</h2>
<p>1. Features from Date (Day of week, month, day of month etc)</p>
<h4><b>TODO</b></h4>
<p>1. Count of Reserved visitors for the nearby restaurant with same genre (TODO - need to add preprocessing for this step)</p>

In [4]:
def create_features(complete_df):
    complete_df['visit_date'] = pd.to_datetime(complete_df['visit_date'])
    complete_df['month'] = complete_df['visit_date'].dt.month
    complete_df['day'] = complete_df['visit_date'].dt.day

create_features(complete_train_data)
create_features(complete_test_data)

In [5]:
week_map = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 
            'Saturday': 5, 'Sunday': 6}
def convert_feature_types(complete_df):
    complete_df.day_of_week = complete_df.day_of_week.map(week_map)
    
convert_feature_types(complete_train_data)
convert_feature_types(complete_test_data)

## Training

In [8]:
independent_cols = ['reserve_visitors', 'day_of_week', 'holiday_flg', 'month', 'day']
train_x = complete_train_data[independent_cols]
train_y = complete_train_data.visitors

rf_reg = RandomForestRegressor()
rf_reg.fit(train_x, train_y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

## Create Submission File

In [28]:
test_x = complete_test_data[independent_cols]
test_x['visitors'] = rf_reg.predict(test_x)

submission_file = pd.DataFrame()
submission_file['air_store_id'] = complete_test_data['air_store_id']
submission_file['visitors'] = test_x['visitors']
submission_file['date'] = pd.to_datetime(complete_test_data['visit_date']).dt.date
submission_file['id'] = submission_file['air_store_id'].map(str) + '_' + submission_file['date'].map(str)
submission_file = submission_file[['id', 'visitors']]
submission_file.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,23.911416
1,air_0164b9927d20bcc3_2017-04-23,23.911416
2,air_0241aa3964b7f861_2017-04-23,23.911416
3,air_0328696196e46f18_2017-04-23,23.911416
4,air_034a3d5b40d5b1b1_2017-04-23,23.911416


In [29]:
submission_file.to_csv('data/processed/submission_1.csv', index=False)